# Feedback

Feedback: Provides strategic points where human judgement is required.
This component implements approval workflows and human-in-the-loop processes for high-risk decisions or complex judgments.

## Feedback with Groq
There is no official feedback system in groq, so we need to make a simple function which asks for user input after drafting a response.

In [ ]:
from groq import Groq
from dotenv import load_dotenv
import os
import json
import asyncio

load_dotenv()
print(os.getenv("GROQ_API_KEY"))

In [ ]:
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

def human_feedback(response: str)-> bool:
    print(f"Generated draft: {response}")
    feedback = input("your feedback")
    return feedback.lower().startswith('y')

def run_conversation_with_human_in_the_loop(prompt: str):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant. You are equipped with human feedback tool, always ask the user whether he/she understood it"
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "human_feedback",
                "description": "Ask for human feedback.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "response":{
                            "type": "string",
                        }
                    },
                    "required": ["response"],
                },
                # "strict": True
            }
        }
    ]
    
    response = client.chat.completions.create(
        model= "meta-llama/llama-4-scout-17b-16e-instruct",
        messages=messages,
        tools=tools,
        tool_choice="auto",
        # max_tokens=max_tokens,
        temperature=0.5,
    )
    print(response.choices[0].message.content)
    tool_calls = response.choices[0].message.tool_calls
    if tool_calls:
        available_tools = {
            "human_feedback": human_feedback
        }
        messages.append(response.choices[0].message)
        # print(messages)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            function_response =  "yes" if available_tools[function_name](**function_args) else "no"
            # print(f"Tool_response: {function_response}")
            messages.append({
                "role": "tool",
                "content": function_response,
                "tool_call_id": tool_call.id
            })
    
    final_response = client.chat.completions.create(
        model = "meta-llama/llama-4-scout-17b-16e-instruct",
        messages=messages,
        # max_tokens=max_tokens,
        temperature=0.7,
    )
    
    return final_response.choices[0].message.content
        

In [54]:
print(run_conversation_with_human_in_the_loop(prompt="help me understand why 2+2 is 4"))

The equation 2+2=4 is a fundamental concept in mathematics, and it's based on the idea of addition. When you add 2 and 2 together, you're essentially counting the total number of items you have.

Imagine you have 2 apples in one hand and 2 apples in the other hand. To find the total number of apples, you count them together: 2 (from one hand) + 2 (from the other hand) = 4 apples.

This equation is a basic example of arithmetic, and it's widely accepted as a true statement. The equals sign (=) indicates that the result of the addition is equal to 4.

Do you understand why 2+2=4? 


Generated draft: The equation 2+2=4 is a fundamental concept in mathematics, and it's based on the idea of addition. When you add 2 and 2 together, you're essentially counting the total number of items you have.  Do you understand why 2+2=4? 
Let me try to explain it in a different way.

Imagine you have 2 apples in one hand, and 2 apples in the other hand. When you put them together, how many apples do you h

## Email
An example of using intelligence, memory, validation, recovery and feedback together.

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional
from groq import Groq
import json
from google import genai
from google.genai import types
from tenacity import retry, stop_after_attempt, retry_if_exception_type, wait_exponential
from google.genai.errors import APIError, ClientError

MODEL = "gemini-2.0-flash"

def human_feedback()-> tuple[bool, str|None]:
    feedback = input()
    if feedback.lower().startswith('n'):
        return (False, feedback)
    else:
        return (True, feedback)
    
@retry(
    stop=stop_after_attempt(2),
    wait=wait_exponential(multiplier=1, max=60, min=1),
    retry=retry_if_exception_type((APIError, ClientError))
) 
def end_convo(contents):
    client = genai.Client()
    response = client.models.generate_content(
        model=MODEL,
        contents=contents
    )
    return response

@retry(
    stop=stop_after_attempt(2),
    wait=wait_exponential(multiplier=1, max=60, min=1),
    retry=retry_if_exception_type((APIError, ClientError))
)
def gen_email(contents, config):
    client = genai.Client()
    response = client.models.generate_content(
        model=MODEL,
        contents=contents,
        config= config
    )
    return response

class EmailDraft(BaseModel):
    to: str = Field(description="reciever's email id")
    subject: Optional[str] = Field(default="No subject", description="subject of the email")
    body: str = Field(description="body of the email")

def run_chat():
    contents = [
        ("system", "You are a email-writter bot. You write relevant and wonderful emails based on user description."),
    ]
    config = types.GenerateContentConfig(
        response_schema=EmailDraft,
        response_mime_type="application/json"
    )
    while(True):
        try:
            if contents[-1][0]!="user":
                prompt = input("User: ")
                contents.append(
                    ("user", prompt)
                )
                print("="*30+" Human Message "+"="*27)
                print(prompt)
                if prompt.lower() in ("bye", "quit", "exit"):
                    response = end_convo(contents)
                    
                    contents.append(
                        ("assistant", response.text)
                    )
                    print("="*30+" AI Message "+"="*30)
                    print(contents[-1][1])
                    break
            
            response = gen_email(contents, config)
            email_data = json.loads(response.text)
            # print(email_data)
            # email_data = json.loads(response.choices[0].message.content)
            contents.append(
                ("assistant", f"""To: {email_data["to"]}\nSubject: {email_data["subject"]}\nBody: {email_data["body"]}\n\nWould you like any other changes?""")
                # "content": f"""To: {email_data["to"]}\nSubject: {email_data["subject"]}\nBody: {email_data["body"]}\n\nWould you like any other changes?"""
            )
            print("="*30+" AI Message "+"="*30)
            print(contents[-1][1])
            
            need_revision, revision = human_feedback()
            
            if not need_revision:
                contents.append(
                    ("user", revision)
                )
                print("="*30+" Human Message "+"="*27)
                print(revision)
                response = end_convo(contents)
                contents.append(
                    ("assistant", response.text)
                )
                print("="*30+" AI Message "+"="*30)
                print(contents[-1][1])
                break
            else:
                contents.append(
                    ("user", f"yes, {revision}")
                )
                print("="*30+" Human Message "+"="*27)
                print(revision)
            
        except Exception as e:
            print("--- EXECUTING FALLBACK ---")
            print("="*30+" AI Message "+"="*30)
            print(f"Looks like I am having a problem generating responses...\n{e}")
            break

In [18]:
run_chat()

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.
Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


============================== Human Message ===========================
write an email to my employer about my resignation. His email id is test1@example.com
============================== AI Message ==============================
To: test1@example.com
Subject: Resignation - [Your Name]
Body: Dear [Employer's Name],

Please accept this email as formal notification that I am resigning from my position as [Your Job Title] at [Company Name], effective [Your Last Day of Employment].

Thank you for the opportunity to work at [Company Name] for the past [Number] years. I have learned and grown professionally during my time here, and I am grateful for the experiences and opportunities I have been given.

I am committed to ensuring a smooth transition during my departure. I am happy to assist in training my replacement and completing any outstanding tasks before my last day.

Please let me know if there is anything else I can do to help with the transition process. I wish you and [Company Nam

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


============================== Human Message ===========================
no
============================== AI Message ==============================
assistantOkay, I'm glad I could help! Let me know if you need anything else in the future.



In [ ]:
draft a mail to a professor for requesting a course named PHI786, highlighting the interest in the course, the email id of the professor is vasudevahn22@iitk.ac.in